# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных:**\
Данные геологоразведки трёх регионов находятся в файлах: 
1) `/datasets/geo_data_0.csv` \
2) `/datasets/geo_data_1.csv` \
3) `/datasets/geo_data_2.csv` 

Признаки:
- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data0 = data0.set_index('id')
data1 = data1.set_index('id')
data2 = data2.set_index('id')

In [4]:
data0.head()

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [6]:
report = data0.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / data0.shape[0]).round(2)*100
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [7]:
data0.duplicated().sum()

0

In [8]:
data1.head()

,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [10]:
data1.duplicated().sum()

0

In [11]:
report = data1.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / data1.shape[0]).round(2)*100
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [12]:
data2.head()

,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746


In [13]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [14]:
report = data2.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / data2.shape[0]).round(2)*100
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


In [15]:
data2.duplicated().sum()

0

Во время данного этапа были импортированы библиотеки, первично изучени датасеты. Они были проверены на пропуски и дубликаты. Столбец `id` был переведён в индексы.

## Обучение и проверка модели

### Работаем с data0

In [16]:
RANDOM_STATE = 42
X0 = data0.drop('product', axis = 1)
y0 = data0['product']
X0_train, X0_valid, y0_train, y0_valid = train_test_split(
        X0, 
        y0, 
        test_size=0.25,
        random_state=RANDOM_STATE
) 

In [17]:
scaler0 = StandardScaler()
X0_train_scaler = scaler0.fit_transform(X0_train)
X0_valid_scaler = scaler0.transform(X0_valid)

In [18]:
model0 = LinearRegression()
model0.fit(X0_train_scaler, y0_train)
predict0 = model0.predict(X0_valid_scaler)

In [19]:
mean0 = predict0.mean()
rmse0 = mean_squared_error(y0_valid, predict0, squared=False)
print(f'Средний запас предсказанного сырья: {mean0}, значение метрики RMSE: {rmse0}')

Средний запас предсказанного сырья: 92.39879990657768, значение метрики RMSE: 37.75660035026169


### Работаем с data1

In [20]:
X1 = data1.drop('product', axis = 1)
y1 = data1['product']
X1_train, X1_valid, y1_train, y1_valid = train_test_split(
        X1, 
        y1, 
        test_size=0.25,
        random_state=RANDOM_STATE
) 

In [21]:
scaler1 = StandardScaler()
X1_train_scaler = scaler1.fit_transform(X1_train)
X1_valid_scaler = scaler1.transform(X1_valid)

In [22]:
model1 = LinearRegression()
model1.fit(X1_train_scaler, y1_train)
predict1 = model1.predict(X1_valid_scaler)

In [23]:
mean1 = predict1.mean()
rmse1 = mean_squared_error(y1_valid, predict1, squared=False)
print(f'Средний запас предсказанного сырья: {mean1}, значение метрики RMSE: {rmse1}')

Средний запас предсказанного сырья: 68.7128780391376, значение метрики RMSE: 0.8902801001028846


### Работаем с data2

In [24]:
X2 = data2.drop('product', axis = 1)
y2 = data2['product']
X2_train, X2_valid, y2_train, y2_valid = train_test_split(
        X2, 
        y2, 
        test_size=0.25,
        random_state=RANDOM_STATE
) 

In [25]:
scaler2 = StandardScaler()
X2_train_scaler = scaler2.fit_transform(X2_train)
X2_valid_scaler = scaler2.transform(X2_valid)

In [26]:
model2 = LinearRegression()
model2.fit(X2_train_scaler, y2_train)
predict2 = model2.predict(X2_valid_scaler)

In [27]:
mean2 = predict2.mean()
rmse2 = mean_squared_error(y2_valid, predict2, squared=False)
print(f'Средний запас предсказанного сырья: {mean2}, значение метрики RMSE: {rmse2}')

Средний запас предсказанного сырья: 94.77102387765939, значение метрики RMSE: 40.145872311342174


Наибольший запас предсказанного сырья у второго месторождения (94.8), на втором месте с незначительной разницей идёт нулевое (92.4). И наименьшие запасы у первого месторождения (68.7).\
Однако именно у первого месторождения модель работает точнее всего (значение метрики RMSE там меньше единицы, а чем ближе значение метрики к нулю, тем точнее модель. В тоже время у других месторождений значение метрики в районе 40)

## Подготовка к расчёту прибыли

In [28]:
#Для разработки выбирают 200 лучших скважин
n = 200
#Бюджет на разработку скважин 10 млрд. рублей
budget = 10 * 10**9
#Доход с единицы продукта
revenue = 450000

In [29]:
#Сколько выделяется денег на одну скважину
budget_one = budget/n
#Сколько нужно единиц продукта, чтобы потрать весь бюджет
volume = budget_one / revenue

In [30]:
volume

111.11111111111111

Значит, если в каждой скважине объем будет не меньше, чем 111.111, то никаких убытков не будет. Но если объем будет меньше, то затраты на бурение не окупятся. Средние запасы в каждом регионе меньше, чем данное число, а значит, либо в них есть 200 скважин с запасом, больше чем 111, либо бурение будет убыточным

## Расчёт прибыли и рисков 

In [31]:
def profit(data, y_true, y_predict):
    data['predict'] = y_predict
    data['true'] = y_true
    data = data.sort_values(by = 'predict', ascending = False)
    data = data.head(200)
    profite = data['true'].sum()*450000 - budget
    return profite

In [32]:
state = np.random.RandomState(12345)

In [33]:
prib0 = []
for i in range(1000):
    subsample = X0_valid.sample(n=500, replace = True, random_state=state)
    prib0.append(profit(subsample, y0_valid, pd.DataFrame(predict0, index = X0_valid.index)))

In [34]:
prib1 = []
for i in range(1000):
    subsample = X1_valid.sample(n=500, replace = True, random_state=state)
    prib1.append(profit(subsample, y1_valid, pd.DataFrame(predict1, index = X1_valid.index)))

In [35]:
prib2 = []
for i in range(1000):
    subsample = X2_valid.sample(n=500, replace = True, random_state=state)
    prib2.append(profit(subsample, y2_valid, pd.DataFrame(predict2, index = X2_valid.index)))

In [36]:
pd.Series(prib0).mean() #Средняя прибыль нулевого региона

406278783.42441905

In [37]:
#Найдём 95% доверительный интервал
lower = pd.Series(prib0).quantile(0.025)
upper = pd.Series(prib0).quantile(0.975)
print(f'95%-доверительный интервал для нулевого региона от {lower} до {upper}')

95%-доверительный интервал для нулевого региона от -117742136.49486831 до 911737050.7514055


In [38]:
len([i for i in prib0 if i<0])/len(prib0)*100 #Доля отрицательных значений в нулевом регионе

6.7

In [39]:
pd.Series(prib1).mean() #Средняя прибыль первого региона

441504277.5922549

In [40]:
#Найдём 95% доверительный интервал
lower = pd.Series(prib1).quantile(0.025)
upper = pd.Series(prib1).quantile(0.975)
print(f'95%-доверительный интервал для первого региона от {lower} до {upper}')

95%-доверительный интервал для первого региона от 35728489.280851334 до 828006639.0043902


In [41]:
len([i for i in prib1 if i<0])/len(prib1)*100 #Доля отрицательных значений в первом регионе

1.6

In [42]:
pd.Series(prib2).mean() #Средняя прибыль второго региона

385213195.91415244

In [43]:
#Найдём 95% доверительный интервал
lower = pd.Series(prib2).quantile(0.025)
upper = pd.Series(prib2).quantile(0.975)
print(f'95%-доверительный интервал для второгозк региона от {lower} до {upper}')

95%-доверительный интервал для второгозк региона от -164785166.1090443 до 888206234.1976783


In [44]:
len([i for i in prib2 if i<0])/len(prib2)*100 #Доля отрицательных значений во втором регионе

7.8

**Вывод**\
Единственный регион с вероятностью убытков меньше 2.5 - это регион 1. В доверительном интервале нет отрицательных значений, значит 95% значений - прибыль, а не убытки. Среднее значение прибыли в этом регионе - 441504277.5922549. Поэтому именно этот регион лучше всего подойдёт для разработки

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован